In [1]:
import os,sys,numpy,rasterio,concurrent,threading,datetime,imageio
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
Image.MAX_IMAGE_PIXELS = None

In [2]:
rawdir = r'C:\Users\GEVENSON\GitHub\NonPerennialStreamPOC\examples\buckhorn\outputs\raw'
gifdir = r'C:\Users\GEVENSON\GitHub\NonPerennialStreamPOC\examples\buckhorn\outputs\gif'
mskdir = r'C:\Users\GEVENSON\GitHub\NonPerennialStreamPOC\examples\buckhorn\outputs\gif\masked'
mapdir = r'C:\Users\GEVENSON\GitHub\NonPerennialStreamPOC\examples\buckhorn\outputs\gif\maps'
strmmsk = r'C:\Users\GEVENSON\GitHub\NonPerennialStreamPOC\examples\buckhorn\inputs\dem\facc_strm_mask.tiff'
if not os.path.isdir(gifdir): os.mkdir(gifdir)
if not os.path.isdir(mskdir): os.mkdir(mskdir)
if not os.path.isdir(mapdir): os.mkdir(mapdir)

In [3]:
dtmin = datetime.datetime(2003, 5, 15, 0, 0, 0)
dtmax = datetime.datetime(2003, 9, 15, 23, 59, 59)

In [ ]:
stream_mask = rasterio.open(strmmsk,'r').read(1).astype(numpy.float32)
stream_mask = numpy.where(stream_mask==1,1,numpy.nan)
lock = threading.Lock()
def mask_streams(args):
    fname = args
    fname_out        = os.path.join(mskdir,f'inund_masked_{fname.split('_')[1]}.tiff')
    if not os.path.isfile(fname_out):
        with rasterio.open(os.path.join(rawdir,fname),'r') as riods:
            inund            = riods.read(1).astype(numpy.float32)
            inund            = numpy.where(inund==1,1,numpy.nan)
            global stream_mask
            with lock:
                inund_msk    = numpy.where(stream_mask==1,inund,numpy.nan)
            inund_msk        = numpy.where(inund_msk==1,1,0)
            fname_out        = os.path.join(mskdir,f'inund_masked_{fname.split('_')[1]}.tiff')
            meta             = riods.meta.copy()
            meta.update({"dtype":rasterio.int8,
                        "nodata":0})
            with rasterio.open(fname_out, "w", **meta) as riods_output:
                riods_output.write(inund_msk,1)
args = [arg for arg in list(os.listdir(rawdir))]
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    executor.map(mask_streams, args)

In [ ]:
outdir = os.path.join(mapdir,'falling_rock')
if not outdir: os.mkdir(outdir)
for fname in os.listdir(mskdir):
    if fname.endswith('.tiff'):
        if not os.path.isfile(os.path.join(outdir,fname.replace('.tiff','.png'))):
            with rasterio.open(os.path.join(mskdir,fname)) as src:
                window = rasterio.windows.Window(col_off=11000, row_off=4500, width=2400, height=2200) # Falling rock
                raster_data = src.read(1,window=window)
                fig, ax = plt.subplots()
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                date = fname.split('_')[2].split('.')[0]
                dt = datetime.datetime(year=int(date[0:4]),month=int(date[4:6]),day=int(date[6:8]))
                ax.set_title(dt.strftime("%b - %d - %Y"))
                ax.imshow(raster_data, cmap='Blues') 
                fig.set_size_inches(5,5)
                plt.savefig(os.path.join(outdir,fname.replace('.tiff','.png')),dpi=900)
                plt.close();

In [10]:
images = []
for fname in os.listdir(mapdir):
    date = fname.split('_')[2].split('.')[0]
    dt = datetime.datetime(year=int(date[0:4]),month=int(date[4:6]),day=int(date[6:8])) 
    if dt >= dtmin and dt <= dtmax:
        images.append(imageio.imread(os.path.join(mapdir,fname)))
imageio.mimsave(os.path.join(gifdir,'output_animation_v2.gif'), images, duration=0.5)

C:\Users\GEVENSON\AppData\Local\Temp\ipykernel_21400\1080657320.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(os.path.join(mapdir,fname)))
